In [1]:
import math

states = ['E', '5', 'I', 'start', 'end']
# Here, I am defining the states of the HMM.
transitions = {
    'start': {'start': float('-inf'), 'E': math.log(1), '5': float('-inf'), 'I':float('-inf'), 'end': float('-inf')},
    'E': {'start': float('-inf'), 'E': math.log(0.9), '5': math.log(0.1), 'I':float('-inf'), 'end': float('-inf')},
    '5': {'start': float('-inf'), 'E':float('-inf'), '5':float('-inf'), 'I': math.log(1.0), 'end': float('-inf')},
    'I': {'start': float('-inf'), 'E':float('-inf'), '5':float('-inf'), 'I': math.log(0.9), 'end': math.log(0.1)},
    'end': {'start': float('-inf'), 'E':float('-inf'), '5':float('-inf'), 'I': float('-inf'), 'end': float('-inf')}
}
# Here I am definign the transition probabilities of the HMM.
# The transition probabilities are defined in log space for numerical stability.
emissions = {
    'start': {'A': float('-inf'), 'C': float('-inf'), 'G': float('-inf'), 'T': float('-inf')},
    'E': {'A': math.log(0.25), 'C': math.log(0.25), 'G': math.log(0.25), 'T': math.log(0.25)},
    '5': {'A': math.log(0.05), 'C': float('-inf'), 'G': math.log(0.95), 'T': float('-inf')},
    'I': {'A': math.log(0.4), 'C': math.log(0.1), 'G': math.log(0.1), 'T': math.log(0.4)},
    'end': {'A': float('-inf'), 'C': float('-inf'), 'G': float('-inf'), 'T': float('-inf')},
}


First of all , I have mentioned all the possible transitions and emissions probabilities. 

This is done to ensure consistency with the Nature primer

In [2]:
# This function takes a sequence of nucleotides and a path (a list of states) as input.
def get_log_prob_of_a_given_path(path, sequence):
    total_log_prob = 0.0
    current_state = 'start'
    
    for i in range(len(path)):
        next_state = path[i]
        nucleotide = sequence[i]

        probt = transitions[current_state][next_state] # Transition probability from current state to next state
        probe = emissions[next_state][nucleotide] # Emission probability of the nucleotide given the next state

        if probt == float('-inf') or probe == float('-inf'): 
            return float('-inf')

        total_log_prob += probt + probe
        current_state = next_state

    total_log_prob += transitions[current_state]['end']# Transition probability from last state to end state
    if total_log_prob == float('-inf'):
        return float('-inf')
    return round(total_log_prob,2) # This function returns the total log probability of the path given the sequence.
# The function rounds the result to 2 decimal places for better readability.


Here, I have defined a function that takes in a path and a sequence as input and outputs the log robability of that path being followed as mentioned in the question.


In [3]:
path = "EEEEEEEEEEEEEEEEEE5IIIIIII"
sequence = "CTTCATGTGAAAGCAGACGTAAGTCA"
print("Log prob of path:", get_log_prob_of_a_given_path(path, sequence))  # Should print -41.22

Log prob of path: -41.22


In [6]:
def viterbi(sequence):
    # Initializing Viterbi matrix and backpointer matrix
    V = [{}]
    backpointer = [{}]

    # I am initializing the starting state probabilities
    for state in states:
        V[0][state] = transitions['start'].get(state, float('-inf')) + emissions[state].get(sequence[0], float('-inf'))
        backpointer[0][state] = 'start'

    # I am runnning the Viterbi algorithm for each subsequent nucleotide
    for t in range(1, len(sequence)):
        V.append({})
        backpointer.append({})

        for state in states:
            max_prob = float('-inf')
            max_state = None

            for prev_state in states:
                prob = V[t - 1][prev_state] + transitions[prev_state].get(state, float('-inf')) + emissions[state].get(sequence[t], float('-inf')) # Emission probability of the nucleotide given the next state
                if prob > max_prob:
                    max_prob = prob
                    max_state = prev_state

            V[t][state] = max_prob
            backpointer[t][state] = max_state

    # Backtrack to find the most likely path
    best_path = []
    max_final_state = max(V[-1], key=V[-1].get)
    best_path.append(max_final_state)

    for t in range(len(sequence) - 2, -1, -1):
        best_path.insert(0, backpointer[t + 1][best_path[0]]) # Backtrack to find the most likely state at time t

    return best_path


Here , I have implemented the Viterbi algorithm to output the most likely path using the backtracking algorithm 

This is an essential step to perform 

In [8]:
sequence = "CTTCATGTGAAAGCAGACGTAAGTCA"
best_path = viterbi(sequence)
print("Most likely path:", best_path)

Most likely path: ['E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E']
